# Gear Design - SpreadSheet 1 Calculator

In [6]:
#Gear Design
import numpy
import pandas as pd
import math
from math import sqrt, pi, ceil, sin
from tkinter import *
from tkinter import ttk

#General
T = 300 #F
Rel = 99
FOS = 2
Bearing_Life = 120 #kh

In [7]:
#Input 
Pinput = 150
Winput = 2000

#Cooling Pump
Ppump = 2
Wpump = 3000 #rpms

#Electrical Gen
Pgen = 53
Wgen = 1800

#Actuators/Motors
Pmotor = {'max':68, 'min':0}
Wmotor = 667

#Layout in
X1 = 12
X2 = 8
Yt = 12
Z1 = 4
Z2 = 8

i=2
angle = math.radians(20)
K =1

#Test Ng
Ng_test = 80

In [8]:
eIC_motor = (Winput/Wmotor)**(1/i)
eIC_pump = (Winput/Wpump)**(1/i)
eIC_gen = (Winput/Wgen)**(1/i)

print("eIC+motor:", eIC_motor)
print("eIC+pump:",eIC_pump)
print("eIC+gen:",eIC_gen, "\n")

Np_eIC_pump = (2*K/((1+(2*eIC_pump))*sin(angle)**2))*(eIC_pump+sqrt(eIC_pump**2+(1+(2*eIC_pump))*sin(angle)**2))
Np_eIC_gen = (2*K/((1+(2*eIC_gen))*sin(angle)**2))*(eIC_gen+sqrt(eIC_gen**2+(1+(2*eIC_gen))*sin(angle)**2))
Np_eIC_motor = (2*K/((1+(2*eIC_motor))*sin(angle)**2))*(eIC_motor+sqrt(eIC_motor**2+(1+(2*eIC_motor))*sin(angle)**2))

# Np_eIC_pump = Ng_test/eIC_pump
# Np_eIC_gen = Ng_test/eIC_gen
# Np_eIC_motor = Ng_test/eIC_motor

print("Np(eIC+pump):",Np_eIC_pump)
print("Np(eIC+gen):",Np_eIC_gen)
print("Np(eIC+motor):",Np_eIC_motor, "\n")

#C = dp+dg/2
#e = dg/dp
#Ni = di*P

A = numpy.matrix([[0.5,0.5,0,0,0,0], [-eIC_motor,1,0,0,0,0], [0,0,0.5,0.5,0,0],
        [0,0,-eIC_pump,1,0,0],[0,0,0,0,0.5,0.5],[0,0,0,0,-eIC_gen,1]])
# print(A, "\n")


B = numpy.matrix([[X1],[0],[sqrt(X2**2+((Yt/2)**2))],[0],[sqrt(X2**2+((Yt/2)**2))],[0],])
# print(B, "\n")

x = numpy.matmul(numpy.linalg.inv(A), B)

x = pd.DataFrame(x,index=(['dp_motor','dg_motor','dp_pump','dg_pump','dp_gen','dg_gen']))

Xmatrix = x[0]
print(Xmatrix, "\n")

print('Rounded dp_motor:',int(round(Xmatrix['dp_motor'].item())))
print('Rounded dg_motor:',int(round(Xmatrix['dg_motor'].item())))
print('Rounded dp_pump:',int(round(Xmatrix['dp_pump'].item())))
print('Rounded dg_pump:',int(round(Xmatrix['dg_pump'].item())))
print('Rounded dp_gen:',int(round(Xmatrix['dp_gen'].item())))
print('Rounded dg_gen:',int(round(Xmatrix['dg_gen'].item())))
print('\n')


P_pump = Np_eIC_pump/Xmatrix['dp_pump']
P_motor = Np_eIC_motor/Xmatrix['dp_motor']
P_gen = Np_eIC_gen/Xmatrix['dp_gen']

print("Pump:", P_pump)
print("Motor:",P_motor)
print("Gen:",P_gen)

print('\n')
# Ng_pump = Np_eIC_pump*(eIC_pump**(1/i))
# print(Ng_pump)
# Ng_motor = Np_eIC_motor*(eIC_motor**(1/i))
# print(Ng_motor)
# Ng_gen = Np_eIC_gen*(eIC_gen**(1/i))
# print(Ng_gen)

Ng_pump=((Np_eIC_pump**2*(sin(angle)**2))-(4*K**2))/((4*K)-(2*Np_eIC_pump*sin(angle)**2))
print(Ng_pump)


eIC+motor: 1.7316179571791195
eIC+pump: 0.816496580927726
eIC+gen: 1.0540925533894598 

Np(eIC+pump): 11.712594898775452
Np(eIC+gen): 12.478195222243716
Np(eIC+motor): 13.82090862040715 

dp_motor     8.786002
dg_motor    15.213998
dp_pump     11.010205
dg_pump      8.989795
dp_gen       9.736660
dg_gen      10.263340
Name: 0, dtype: float64 

Rounded dp_motor: 9
Rounded dg_motor: 15
Rounded dp_pump: 11
Rounded dg_pump: 9
Rounded dp_gen: 10
Rounded dg_gen: 10


Pump: 1.0637944293708566
Motor: 1.5730600905014944
Gen: 1.2815683942875378


9.563293688641679


# FOS Calculator from class 4/25 (complete)

### Planning on making this an interactive GUI for multiple sections (motor, pump, gen ect)

### Numbers for Gear Setup

In [9]:


from math import pi,sqrt,sin,cos,radians
import math
Rel = .99
P_life = 10**8
St_p = 75000 #31350
St_g = 75000 #28260




Kt = 1 #T<250 F, Kt=1
Kr = 1

Np = 24
Ng= 96
p = 12
np = 2000
dp= Np/p
V = (pi*dp*np)/12
print('V',V)
H = 68 #hp
gr =Ng/Np
theta = radians(20)
Yn_p = 1.3558*P_life**-0.0178
Yn_g = 1.3558*((10**8/(gr))**-0.0178)

print('Ynp:',round(Yn_p,3),'Yng:',round(Yn_g,3))


Wt = 33000*H/V
print('Wt:',round(Wt,3))

K_o = 1

val = numpy.matrix([[12,13,14,15,16,17,18,19,20,21,22,24,26,28,30,34,38,43,50,60,75,100,150,300,400],
             [0.245,0.261,0.277,0.290,0.296,0.303,0.309,0.314,0.322,0.328,0.331,0.337,0.346,0.353,0.359,0.371,0.384,0.397,0.409,0.422,0.435,0.447,0.460,0.472,0.480]])
Lewis_FF = pd.DataFrame(val.T,columns=['Number of teeth', 'Y'])

Qv=12
B = 0.25*(12-Qv)**(2/3)
print('B',B)
A = 50+56*(1-B)
print('A',A)
K_v = ((A+sqrt(V))/A)**B
print('K_v',K_v)
Fw = 3.5
try:
    Yp = Lewis_FF[Lewis_FF['Number of teeth']==Np]['Y'].item()
except ValueError:
    Yp = numpy.interp(Np,[75,100],[0.460,0.472])
    print('interpolate Yp:',Yp)
    
try:
    Yg = Lewis_FF[Lewis_FF['Number of teeth']==Ng]['Y'].item()
except ValueError:
    Yg = numpy.interp(Ng,[75,100],[0.435,0.447])
    print('interpolate Yg:', Yg)


K_sp = 1.192*(Fw*sqrt(Yp)/p)**0.0535
K_sg = 1.192*(Fw*sqrt(Yg)/p)**0.0535


print('Ksp:',round(K_sp,3),'Ksg:',round(K_sg,3))

Cmc = 1
Cpf = (Fw/(10*dp))-0.0375 + (0.0125*Fw) #0.0695
print('Cpf',Cpf)
Cpm = 1
A1 =0.127
B1 = 0.0158
C1 = -0.930*10**-4
Cma = (A1+(B1*Fw)+(C1*Fw**2)) #eqn 14-34 0.15
print('Cma',Cma)
Ce = 1 #No adjustments
Km = 1+Cmc*(Cpf*Cpm+Cma*Ce) #1.22
print ('Km:',round(Km,3))
Kb = 1 #solid

Jp = 0.30
Jg = 0.40
sigma_p = Wt*K_o*K_v*K_sp*(p/Fw)*(Km*Kb/Jp) 
print('Sigma_p',sigma_p)
sigma_g = Wt*K_o*K_v*K_sg*(p/Fw)*(Km*Kb/Jg) 
print('Sigma_g',sigma_g)

Sf_p = (St_p*Yn_p/(Kt*Kr))/sigma_p #5.62
print('Sf_p:',round(Sf_p,3))
Sf_g = (St_g*Yn_g/(Kt*Kr))/sigma_g #6.82
print('Sf_g:',round(Sf_g,3))


Sc_p = 275000# 106400 # fig 4-5 contact endurance strength 322Hr + 29100 psi
Sc_g = 275000 #psi

Mn=1
Cp = 2300 #p.749 
Cf = 1 
I = (cos(theta)*sin(theta)/2*Mn)*(gr/(gr+1)) #=0.121  gr-(Gear ratio)
print('I',I)
sigma_contact_p = Cp*(Wt*K_o*K_v*K_sp*(Km/(dp*Fw))*(Cf/I))**0.5 #= 70360
print('sigma_contact_p:',round(sigma_contact_p,3))
sigma_contact_g = Cp*(Wt*K_o*K_v*K_sg*(Km/(dp*Fw))*(Cf/I))**0.5 #= 70660
print('sigma_contact_g:',round(sigma_contact_g,3))

Zn_p = 0.948
Zn_g = 1.4458*(10**8/gr)**-0.023
Sh_p = Sc_p*Zn_p/(Kt*Kr)/sigma_contact_p
print('Sh_p:',round(Sh_p,3))
Sh_g = Sc_g*Zn_g/(Kt*Kr)/sigma_contact_g
print('Sh_g:',round(Sh_g,3))
print('\n')


#Sfp = 5.62
#Sfg = 6.82
Sf_p = (St_p*Yn_p/(Kt*Kr))/sigma_p #5.62
#print('Bending factor of safety for pinion Sf_p:',round(Sf_p,2))
Sf_g = (St_g*Yn_g/(Kt*Kr))/sigma_g #6.82
#print('Bending factor of safety for gear Sf_g:',round(Sf_g,2))

#Pinion
#Sfp = 5.62 
#Shp**2=2.86
print('Wear factor of safety for pinion Sh_p^2:',round(Sh_p**2,2),'\n','Bending factor of safety for pinion Sf_p:',round(Sf_p,2))
print('\n')
#Gear
#Sfg = 6.82
#Shg**2 = 2.36
print('Wear factor of safety for gear Sh_g^2:',round(Sh_g**2,2),'\n','Bending factor of safety for gear Sf_g:',round(Sf_g,2))

V 1047.1975511965977
Ynp: 0.977 Yng: 1.001
Wt: 2142.862
B 0.0
A 106.0
K_v 1.0
interpolate Yg: 0.44508000000000003
Ksp: 1.084 Ksg: 1.092
Cpf 0.18125
Cma 0.18116075
Km: 1.362
Sigma_p 36166.463373539074
Sigma_g 27327.438427615685
Sf_p: 2.026
Sf_g: 2.748
I 0.12855752193730788
sigma_contact_p: 136391.441
sigma_contact_g: 136899.836
Sh_p: 1.911
Sh_g: 1.963


Wear factor of safety for pinion Sh_p^2: 3.65 
 Bending factor of safety for pinion Sf_p: 2.03


Wear factor of safety for gear Sh_g^2: 3.85 
 Bending factor of safety for gear Sf_g: 2.75
